In [2]:
import random
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense , Activation
from tensorflow.keras.optimizers import RMSprop

In [4]:
filePath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filePath ,'rb').read().decode(encoding='utf-8').lower()


1115394/1115394 [==============================] - 0s 0us/step


In [5]:
text = text[300000:800000]
characters = sorted(set(text))
char_to_index = dict((c,i) for i,c  in enumerate(characters))
index_to_char = dict((i,c) for i,c  in enumerate(characters))

In [6]:
seq_length = 40
step_size = 3
sentences = []
next_characters =[]

for i  in range(0 , len(text)-seq_length,step_size):
  sentences.append(text[i:i+seq_length])
  next_characters.append(text[i+seq_length])

In [11]:
x = np.zeros((len(sentences), seq_length,
              len(characters)), dtype=np.bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)

for i,sentence in enumerate(sentences):
  for t,character in enumerate(sentence) :
    x[i,t,char_to_index[character]]=1
  y[i,char_to_index[next_characters[i]]]=1


<ipython-input-11-f868b05adf80>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)
<ipython-input-11-f868b05adf80>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)


In [13]:
  model = Sequential()
  model.add(LSTM(128,input_shape=(seq_length, len(characters))))
  model.add(Dense(len(characters)))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01))
  model.fit(x,y,batch_size=256,epochs=4)
  model.save('textgenerator.model')


Epoch 1/4
651/651 [==============================] - 128s 193ms/step - loss: 2.1584
Epoch 2/4
651/651 [==============================] - 130s 200ms/step - loss: 1.7281
Epoch 3/4
651/651 [==============================] - 124s 191ms/step - loss: 1.6018
Epoch 4/4
651/651 [==============================] - 124s 190ms/step - loss: 1.5309


In [14]:
model = tf.keras.models.load_model('textgenerator.model')
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [22]:
def generate_text(length,temperture):
  start_index = random.randint(0,len(text)-seq_length -1)
  generated = ''
  sentence = text[start_index:start_index + seq_length]
  generated += sentence
  for i in range(length):
    x = np.zeros((1,seq_length,len(characters)))
    for t,character in enumerate(sentence):
      x[0,t,char_to_index[character]]=1
    predictions = model.predict(x,verbose=0)[0]
    next_index = sample(predictions , temperture)
    next_character = index_to_char[next_index]
    generated += next_character
    sentence = sentence[1:]+next_character
  return generated

In [24]:
print('_________________________0.2______')
print(generate_text(300, 0.2))
print('_________________________0.4______')
print(generate_text(300, 0.4))
print('_________________________0.5______')
print(generate_text(300, 0.5))
print('_________________________0.6______')
print(generate_text(300, 0.6))
print('_________________________0.7______')
print(generate_text(300, 0.7))
print('_________________________0.8______')
print(generate_text(300, 0.8))

_________________________0.2______
igh, my lord?

king henry vi:
not for my lord, and i will strengther son.

romeo:
and the regense the seaver to the bears,
and i see is the bears, with the recouss
and in the romeo, the father and the words,
and i will i see the seaver to the father,
and i see to the counting to the breath,
and i see to mer to the father with the cause
to
_________________________0.4______
d by the way
and lost all my money?

autolycus:
why, so shall stret shall see but of the bastress
on the strung to the redoth of the black.

romeo:
and i confess in the baster of these is the rear.

clown:
we was the world, i have say for the heaven,
as when i streaked to be to my bearty to the cause,
to my lord, to may thou streaked to t
_________________________0.5______
rrish riddles sort not with this place.

romeo:
now i see the repuse to the grave sire every speak.

semplo:
a duke of arms? god such such this stands true heart and all all thine heart shall i thereon,
to thy bro